In [110]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import seaborn as sns

In [111]:
df=pd.read_csv('movies.csv',usecols=['original_title','actors','imdb_votes','runtime','imdb_rating','wins_nominations'])

In [112]:
df.head()

,original_title,runtime,imdb_rating,imdb_votes,actors,wins_nominations
0,Uri: The Surgical Strike,138,8.4,35112,Vicky Kaushal|Paresh Rawal|Mohit Raina|Yami Ga...,4 wins
1,Battalion 609,131,4.1,73,Vicky Ahuja|Shoaib Ibrahim|Shrikant Kamat|Elen...,NaN
2,The Accidental Prime Minister,112,6.1,5549,Anupam Kher|Akshaye Khanna|Aahana Kumra|Atul S...,NaN
3,Why Cheat India,121,6.0,1891,Emraan Hashmi|Shreya Dhanwanthary|Snighdadeep ...,NaN
4,Evening Shadows,102,7.3,280,Mona Ambegaonkar|Ananth Narayan Mahadevan|Deva...,17 wins & 1 nomination


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1629 entries, 0 to 1628
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   original_title    1629 non-null   object 
 1   runtime           1629 non-null   object 
 2   imdb_rating       1629 non-null   float64
 3   imdb_votes        1629 non-null   int64  
 4   actors            1624 non-null   object 
 5   wins_nominations  707 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 76.5+ KB


In [114]:
df['wins_nominations'] = df['wins_nominations'].astype(str)
df['NW']=df['wins_nominations'].str.split(' ',expand=True)[0]

In [115]:
import numpy as np
df['NW'] = df['NW'].replace({'nan': np.nan,'Nominated':np.nan})


In [116]:
df['NW'] = df['NW'].fillna(0)


In [117]:
df['NW'] = df['NW'].astype('int32')


In [118]:
df['NW']

,NW
0,4
1,0
2,0
3,0
4,17
...,...
1624,0
1625,0
1626,0
1627,0


In [119]:
df.drop(columns='wins_nominations',inplace=True)

In [120]:
df['imdb_votes_per_rating']=df['imdb_votes']/df['imdb_rating']

In [121]:

# Replace '\\N' with actual NaN
df['runtime'] = df['runtime'].replace('\\N', np.nan)

# Convert to numeric (forcing errors to NaN), then fill and cast
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')
df['runtime'] = df['runtime'].fillna(df['runtime'].mean()).astype('int32')


In [122]:
df['runtime']

,runtime
0,138
1,131
2,112
3,121
4,102
...,...
1624,148
1625,146
1626,132
1627,136


In [123]:
df.head()

,original_title,runtime,imdb_rating,imdb_votes,actors,NW,imdb_votes_per_rating
0,Uri: The Surgical Strike,138,8.4,35112,Vicky Kaushal|Paresh Rawal|Mohit Raina|Yami Ga...,4,4180.000000
1,Battalion 609,131,4.1,73,Vicky Ahuja|Shoaib Ibrahim|Shrikant Kamat|Elen...,0,17.804878
2,The Accidental Prime Minister,112,6.1,5549,Anupam Kher|Akshaye Khanna|Aahana Kumra|Atul S...,0,909.672131
3,Why Cheat India,121,6.0,1891,Emraan Hashmi|Shreya Dhanwanthary|Snighdadeep ...,0,315.166667
4,Evening Shadows,102,7.3,280,Mona Ambegaonkar|Ananth Narayan Mahadevan|Deva...,17,38.356164


In [154]:
new_df=df[['imdb_rating','runtime','imdb_votes','imdb_votes_per_rating','NW']]

In [155]:
x=new_df.iloc[:,0:5]
y=new_df.iloc[:,-1]
x

,imdb_rating,runtime,imdb_votes,imdb_votes_per_rating,NW
0,8.4,138,35112,4180.000000,4
1,4.1,131,73,17.804878,0
2,6.1,112,5549,909.672131,0
3,6.0,121,1891,315.166667,0
4,7.3,102,280,38.356164,17
...,...,...,...,...,...
1624,4.9,148,278,56.734694,0
1625,3.0,146,133,44.333333,0
1626,6.1,132,13,2.131148,0
1627,7.4,136,38,5.135135,0


In [156]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_scaled=sc.fit_transform(x)
x_scaled_df = pd.DataFrame(x_scaled)
x_clean = x_scaled_df.dropna()
y_clean = y[x_clean.index]

In [158]:
np.mean(cross_val_score(LogisticRegression(),x_clean,y_clean,scoring='accuracy',cv=5))


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modu

np.float64(0.6487654320987654)